In [ ]:
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Sequence



from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool, BaseTool
from langchain_ollama import ChatOllama



from langgraph.managed import IsLastStep
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.prebuilt import (ToolNode, create_react_agent, tools_condition)



from tools import tavily_search, random_number_maker, text_to_image

In [ ]:
tools = [tavily_search, random_number_maker, text_to_image]
tools_node = ToolNode(tools=tools)



class ReActAgentState(BaseModel):
	"""ReAct Agent with structured response format."""
	user_prompt: str = Field(description="The original question provided by the user.")
	thought: str = Field(description="Logical reasoning before executing an action")
	action: str = Field(description=f"The action to be taken, chosen from available tools {', '.join([tool.name for tool in tools])}.")
	action_input:str = Field(description="The required input for the action.") 
	observation: str = Field(description="The outcome of executing the action.")



class DataProcessAgentState(BaseModel):
	pass 



class TrainingProcessAgentState(BaseModel):
	pass



class State(BaseModel):
	react_agent_state: Annotated[Sequence[ReActAgentState], add_messages]
	data_process_agent_state: Annotated[Sequence[DataProcessAgentState], add_messages]
	is_last_step: IsLastStep
	remaining_steps: int

In [3]:
checkpointer = MemorySaver()

In [4]:
model = ChatOllama(model="llama3.2:1b-instruct-fp16", temperature=0.1, num_predict="2048")
react_agent = create_react_agent(
	name="react_agent", 
	model=model, 
    tools=tools, 
	response_format=ReActAgentState,
	checkpointer=checkpointer, 
	prompt=SystemMessage(content="You are a helpful assistant!"),
)

In [5]:
from IPython.display import Image, display
display(Image(react_agent.get_graph().draw_mermaid_png()))

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 502.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

inputs = {"messages": [
	SystemMessage("You are helpful assistant, if the tool get error, you can tell that you don't know the answer"), 
	HumanMessage("what is the weather in sf")
]}
for s in graph.stream(inputs, stream_mode="values"):
	message = s["messages"][-1]
	if isinstance(message, tuple):
		print(message)
	else:
		message.pretty_print()

In [ ]:
(-2.1)

In [ ]:
prompt = [HumanMessage("What is the capital of France?")]

In [ ]:
model.invoke(prompt)

In [ ]:
system_msg = SystemMessage("""You are a helpful assistant that responds to questions with three exclamation marks.""")
human_msg = HumanMessage('What is the capital of France?')
model.invoke([system_msg, human_msg])

In [ ]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template("""Answer the question based on the context below. If the question cannot be answered using the information provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

template.invoke({"context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperform their smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers can tap into these models through Hugging Face's `transformers` library, or by utilizing OpenAI and Cohere's offerings through the `openai` and `cohere` libraries, respectively.""",
	"question": "Which model providers offer LLMs?"
})

In [ ]:
completion = model.invoke(prompt)

In [ ]:
from pydantic import BaseModel
class AnswerWithJustification(BaseModel):
	'''An answer to the user's question along with justification for the answer.'''
	answer: str
	'''The answer to the user's question'''
	justification: str
	'''Justification for the answer'''

In [ ]:
structured_llm = model.with_structured_output(AnswerWithJustification)
structured_llm

In [ ]:
structured_llm.invoke("What weighs more, a pound of bricks or a pound of feathers")